# Agent-Based Model of Evans 1997 using Agents.jl (Claude)
Tomás Aragón

This notebook replicates the Evans 1997 migraine cost-effectiveness analysis using an agent-based (microsimulation) approach with [Agents.jl](https://juliadynamics.github.io/Agents.jl/stable/). Each agent is a migraine patient who walks through the decision tree stochastically, accumulating costs and receiving a utility outcome.

## Conceptual comparison

| Approach | Method | Output |
|----------|--------|--------|
| **DecisionProgramming.jl** | Analytical: computes exact expected values via influence diagram | Exact E[Cost], E[Utility] |
| **Agents.jl (this notebook)** | Simulation: N patients traverse the tree stochastically | Monte Carlo estimates of E[Cost], E[Utility] |

With enough agents, the ABM converges to the analytical solution. The ABM approach additionally gives us the full distribution of individual-level outcomes, which enables probabilistic sensitivity analysis and heterogeneity extensions.

Sources:
- Evans, K. W., et al. "Economic Evaluation of Oral Sumatriptan Compared with Oral Caffeine/Ergotamine for Migraine." *PharmacoEconomics* 12, no. 5 (1997): 565–77.
- Briggs, A. H., K. Claxton, and M. Sculpher. *Decision Modelling for Health Economic Evaluation.* Oxford Univ. Press, 2011.

## Setup and model parameters

In [9]:
using Agents
using Random
using Statistics
using DataFrames
using Printf    

### Model parameters

These are the same 14 parameters (4 costs, 4 utilities, 6 probabilities) from the Evans 1997 decision tree.

In [10]:
# Evans 1997 model parameters stored in a NamedTuple
params = (
    # Costs (Canadian dollars)
    c_sumatriptan = 16.10,
    c_caffeine    = 1.32,
    c_ed          = 63.16,
    c_hospital    = 1093.0,

    # Utilities (quality-of-life weights)
    u_relief_norecurrence = 1.0,
    u_relief_recurrence   = 0.9,
    u_norelief_endures    = -0.30,
    u_norelief_ed         = 0.1,

    # Probabilities
    p_relief_sumatriptan  = 0.558,   # P(relief | sumatriptan)
    p_relief_caffeine     = 0.379,   # P(relief | caffeine)
    p_norecurrence_sumatriptan = 0.594, # P(no recurrence | relief, sumatriptan)
    p_norecurrence_caffeine   = 0.703, # P(no recurrence | relief, caffeine)
    p_endures_norelief    = 0.92,    # P(endures | no relief) — both arms
    p_relief_ed           = 0.998,   # P(relief | ED visit) — both arms
)

(c_sumatriptan = 16.1, c_caffeine = 1.32, c_ed = 63.16, c_hospital = 1093.0, u_relief_norecurrence = 1.0, u_relief_recurrence = 0.9, u_norelief_endures = -0.3, u_norelief_ed = 0.1, p_relief_sumatriptan = 0.558, p_relief_caffeine = 0.379, p_norecurrence_sumatriptan = 0.594, p_norecurrence_caffeine = 0.703, p_endures_norelief = 0.92, p_relief_ed = 0.998)

## Define the agent type

Each agent represents a migraine patient. Since there is no spatial interaction between patients (each walks through the tree independently), we use `NoSpaceAgent`. The agent records:
- `treatment`: which arm (`:sumatriptan` or `:caffeine`)
- `cost`: total cost accumulated along the path
- `utility`: quality-of-life outcome
- `path`: which leaf node (A–J) the agent ended at

In [11]:
@agent struct Patient(NoSpaceAgent)
    treatment::Symbol      # :sumatriptan or :caffeine
    cost::Float64 = 0.0
    utility::Float64 = 0.0
    path::Symbol = :unresolved
end

## Decision tree traversal

The `agent_step!` function implements the Evans 1997 decision tree. Each agent traverses the tree in a single step by sampling from the conditional probabilities at each chance node.

```
Treatment ─┬─ Relief? ─── Yes ─┬─ Recurrence? ─── No  → Leaf A/F (utility = 1.0)
           │                   │                  Yes → Leaf B/G (utility = 0.9, +drug cost)
           │                   │
           └─ Relief? ─── No  ─┬─ Endures?  ──── Yes → Leaf C/H (utility = -0.3)
                               │
                               └─ ED visit  ────┬─ Relief     → Leaf D/I (utility = 0.1)
                                                └─ Hospital   → Leaf E/J (utility = -0.3)
```

In [12]:
function agent_step!(agent, model)
    # Skip agents already resolved
    agent.path != :unresolved && return

    p = abmproperties(model)
    rng = abmrng(model)

    # Treatment-specific parameters
    if agent.treatment == :sumatriptan
        p_relief = p.p_relief_sumatriptan
        p_norecurrence = p.p_norecurrence_sumatriptan
        c_drug = p.c_sumatriptan
    else
        p_relief = p.p_relief_caffeine
        p_norecurrence = p.p_norecurrence_caffeine
        c_drug = p.c_caffeine
    end

    # Initial drug cost
    agent.cost = c_drug

    # Chance node: Relief?
    if rand(rng) < p_relief
        # Relief branch
        # Chance node: Recurrence?
        if rand(rng) < p_norecurrence
            # No recurrence → Leaf A/F
            agent.utility = p.u_relief_norecurrence
            agent.path = agent.treatment == :sumatriptan ? :A : :F
        else
            # Recurrence → relieved with 2nd dose → Leaf B/G
            agent.cost += c_drug   # 2nd dose
            agent.utility = p.u_relief_recurrence
            agent.path = agent.treatment == :sumatriptan ? :B : :G
        end
    else
        # No relief branch
        # Chance node: Endures or ED?
        if rand(rng) < p.p_endures_norelief
            # Endures attack → Leaf C/H
            agent.utility = p.u_norelief_endures
            agent.path = agent.treatment == :sumatriptan ? :C : :H
        else
            # Emergency department
            agent.cost += p.c_ed
            # Chance node: ED relief or hospitalization?
            if rand(rng) < p.p_relief_ed
                # ED relief → Leaf D/I
                agent.utility = p.u_norelief_ed
                agent.path = agent.treatment == :sumatriptan ? :D : :I
            else
                # Hospitalization → Leaf E/J
                agent.cost += p.c_hospital
                agent.utility = p.u_norelief_endures
                agent.path = agent.treatment == :sumatriptan ? :E : :J
            end
        end
    end
end

agent_step! (generic function with 1 method)

## Model constructor

This function creates the ABM for a given treatment arm and number of patients.

In [13]:
function evans_abm(;
    treatment::Symbol,
    n_patients::Int = 100_000,
    seed::Int = 42,
    params = params
)
    model = StandardABM(
        Patient;
        agent_step!,
        properties = params,
        rng = Random.Xoshiro(seed)
    )
    for _ in 1:n_patients
        add_agent!(model; treatment = treatment)
    end
    return model
end

evans_abm (generic function with 1 method)

## Run the simulation

We run each treatment arm separately with 1,000,000 patients. Since the decision tree is traversed in a single step, we only need 1 step.

In [14]:
N = 1_000_000

# --- Sumatriptan arm ---
model_d1 = evans_abm(treatment = :sumatriptan, n_patients = N, seed = 123)
step!(model_d1, 1)

# --- Caffeine/Ergotamine arm ---
model_d2 = evans_abm(treatment = :caffeine, n_patients = N, seed = 456)
step!(model_d2, 1)

println("Simulation complete: $(N) patients per arm")

Simulation complete: 1000000 patients per arm


## Extract results

In [15]:
# Helper: collect agent data into vectors
function collect_results(model)
    agents = collect(allagents(model))
    costs     = [a.cost for a in agents]
    utilities = [a.utility for a in agents]
    paths     = [a.path for a in agents]
    return (; costs, utilities, paths)
end

res_d1 = collect_results(model_d1)
res_d2 = collect_results(model_d2);

### Path frequencies

Compare simulated path frequencies against analytical probabilities from the decision tree.

In [16]:
function path_table(paths, label)
    counts = Dict{Symbol,Int}()
    for p in paths
        counts[p] = get(counts, p, 0) + 1
    end
    n = length(paths)
    println("\n=== $label ===")
    println("Path   Count      Simulated     Analytical")
    for k in sort(collect(keys(counts)))
        sim_p = counts[k] / n
        @printf("  %s    %7d    %.6f\n", k, counts[k], sim_p)
    end
end

path_table(res_d1.paths, "Sumatriptan (D1)")
path_table(res_d2.paths, "Caffeine/Ergotamine (D2)")


=== Sumatriptan (D1) ===
Path   Count      Simulated     Analytical
  A     331735    0.331735
  B     226289    0.226289
  C     406681    0.406681
  D      35234    0.035234
  E         61    0.000061

=== Caffeine/Ergotamine (D2) ===
Path   Count      Simulated     Analytical
  F     267283    0.267283
  G     112014    0.112014
  H     570987    0.570987
  I      49633    0.049633
  J         83    0.000083


### Cost-effectiveness analysis

In [17]:
# Expected values from simulation
EC_D1 = mean(res_d1.costs)
EU_D1 = mean(res_d1.utilities)
EC_D2 = mean(res_d2.costs)
EU_D2 = mean(res_d2.utilities)

# Incremental CEA
ΔC = EC_D1 - EC_D2
ΔU = EU_D1 - EU_D2
ICER = (ΔC / ΔU) * 365   # annualize: 24h time horizon

println("=== ABM Cost-Effectiveness Analysis (N = $N per arm) ===")
println("")
println("Strategy           E[Cost]     E[Utility]")
@printf("Caffeine/Ergot     %8.4f    %8.4f\n", EC_D2, EU_D2)
@printf("Sumatriptan        %8.4f    %8.4f\n", EC_D1, EU_D1)
println("")
@printf("Incremental Cost:    %8.4f\n", ΔC)
@printf("Incremental Utility: %8.4f\n", ΔU)
@printf("ICER: \$%.0f Can/QALY\n", ICER)

=== ABM Cost-Effectiveness Analysis (N = 1000000 per arm) ===

Strategy           E[Cost]     E[Utility]
Caffeine/Ergot       4.6986      0.2017
Sumatriptan         22.0392      0.4169

Incremental Cost:     17.3405
Incremental Utility:   0.2152
ICER: $29417 Can/QALY


### Comparison with analytical (exact) results

In [18]:
# Analytical reference values from DecisionProgramming.jl / rdecision
EC_D1_exact = 22.058057
EU_D1_exact = 0.4168609
EC_D2_exact = 4.714972
EU_D2_exact = 0.2012760
ICER_exact  = 29363.0

println("=== Comparison: ABM vs Analytical ===")
println("")
println("                     ABM          Analytical     Difference")
@printf("E[C|Sumatriptan]     %8.4f     %8.4f       %+.4f\n", EC_D1, EC_D1_exact, EC_D1 - EC_D1_exact)
@printf("E[U|Sumatriptan]     %8.4f     %8.4f       %+.6f\n", EU_D1, EU_D1_exact, EU_D1 - EU_D1_exact)
@printf("E[C|Caffeine]        %8.4f     %8.4f       %+.4f\n", EC_D2, EC_D2_exact, EC_D2 - EC_D2_exact)
@printf("E[U|Caffeine]        %8.4f     %8.4f       %+.6f\n", EU_D2, EU_D2_exact, EU_D2 - EU_D2_exact)
@printf("ICER                 %8.0f     %8.0f       %+.0f\n", ICER, ICER_exact, ICER - ICER_exact)

=== Comparison: ABM vs Analytical ===

                     ABM          Analytical     Difference
E[C|Sumatriptan]      22.0392      22.0581       -0.0189
E[U|Sumatriptan]       0.4169       0.4169       +0.000035
E[C|Caffeine]          4.6986       4.7150       -0.0163
E[U|Caffeine]          0.2017       0.2013       +0.000462
ICER                    29417        29363       +54


## Using Agents.jl `run!` with data collection

Agents.jl provides a built-in `run!` function that handles stepping and data collection in one call. Here we demonstrate this approach, collecting per-agent cost, utility, and path at the end of the simulation.

In [19]:
# Re-create models
model_d1b = evans_abm(treatment = :sumatriptan, n_patients = N, seed = 789)
model_d2b = evans_abm(treatment = :caffeine, n_patients = N, seed = 101)

# Define what agent data to collect
adata = [:cost, :utility, :path]

# Run for 1 step and collect
df_d1, _ = run!(model_d1b, 1; adata)
df_d2, _ = run!(model_d2b, 1; adata)

# Filter to final step (time == 1)
df_d1_final = filter(:time => ==(1), df_d1)
df_d2_final = filter(:time => ==(1), df_d2)

@printf("run!() approach — E[C|D1] = %.4f, E[U|D1] = %.4f\n",
    mean(df_d1_final.cost), mean(df_d1_final.utility))
@printf("run!() approach — E[C|D2] = %.4f, E[U|D2] = %.4f\n",
    mean(df_d2_final.cost), mean(df_d2_final.utility))

run!() approach — E[C|D1] = 22.0601, E[U|D1] = 0.4169
run!() approach — E[C|D2] = 4.7011, E[U|D2] = 0.2004


## Extension: Monte Carlo confidence intervals

One advantage of the ABM/microsimulation approach over the analytical solution is that we get the full distribution of individual outcomes. We can compute confidence intervals for the ICER using bootstrap or directly from the standard errors.

In [20]:
# Standard errors
se_cost_d1 = std(res_d1.costs) / sqrt(N)
se_util_d1 = std(res_d1.utilities) / sqrt(N)
se_cost_d2 = std(res_d2.costs) / sqrt(N)
se_util_d2 = std(res_d2.utilities) / sqrt(N)

println("=== Monte Carlo Standard Errors ===")
@printf("SE[E[C|Sumatriptan]] = %.4f\n", se_cost_d1)
@printf("SE[E[U|Sumatriptan]] = %.6f\n", se_util_d1)
@printf("SE[E[C|Caffeine]]    = %.4f\n", se_cost_d2)
@printf("SE[E[U|Caffeine]]    = %.6f\n", se_util_d2)

println("\n=== 95%% Confidence Intervals ===")
@printf("E[C|Sumatriptan]: %.4f ± %.4f\n", EC_D1, 1.96 * se_cost_d1)
@printf("E[U|Sumatriptan]: %.4f ± %.6f\n", EU_D1, 1.96 * se_util_d1)
@printf("E[C|Caffeine]:    %.4f ± %.4f\n", EC_D2, 1.96 * se_cost_d2)
@printf("E[U|Caffeine]:    %.4f ± %.6f\n", EU_D2, 1.96 * se_util_d2)

=== Monte Carlo Standard Errors ===
SE[E[C|Sumatriptan]] = 0.0157
SE[E[U|Sumatriptan]] = 0.000615
SE[E[C|Caffeine]]    = 0.0173
SE[E[U|Caffeine]]    = 0.000608

=== 95%% Confidence Intervals ===
E[C|Sumatriptan]: 22.0392 ± 0.0307
E[U|Sumatriptan]: 0.4169 ± 0.001205
E[C|Caffeine]:    4.6986 ± 0.0338
E[U|Caffeine]:    0.2017 ± 0.001191


## Summary

This notebook demonstrates that the Evans 1997 decision tree can be faithfully replicated as an agent-based microsimulation using Agents.jl. Key observations:

1. **Convergence**: With 1M agents per arm, the ABM estimates converge to the analytical values from DecisionProgramming.jl and R `rdecision`.

2. **Advantages of the ABM approach**:
   - Provides the **full distribution** of individual outcomes, not just expected values
   - Naturally produces **confidence intervals** via Monte Carlo standard errors
   - Easy to extend with **patient heterogeneity** (e.g., age-dependent probabilities, comorbidities)
   - Straightforward to add **multi-period dynamics** (e.g., repeated migraine episodes over time)
   - Can incorporate **agent interactions** (e.g., resource constraints, waiting times at ED)

3. **Trade-off**: The analytical approach (DecisionProgramming.jl) gives exact answers instantly; the ABM requires many agents for precision but offers richer output.